<a href="https://colab.research.google.com/github/WaruCodes/Deep_Learning-Defect-prediction-framework-using-CodeBERT-and-GAT/blob/main/Read_the_Black.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required libraries
!pip install -q duckdb polars fastparquet datasets tensorflow==2.19.0 lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.8 MB/s eta 0:00:00


In [3]:
import pandas as pd

file_path = '/content/drive/MyDrive/defectors/line_bug_prediction_splits/random/train.parquet.gzip'

df = pd.read_parquet(file_path)
print(df.head())

                   datetime                                    commit  \
0 2016-08-11 14:07:40-01:00  01b498ec5109da22bf1b79d86efaecf45426ad51   
0 2020-07-24 17:47:46-01:00  007bc310840d9cd5b37983a0c6ba82bd9e551c26   
0 2014-07-16 03:51:12-01:00  0786e84a33155ebc8d8d3502e3a7f3060b86a4ec   
0 2019-08-15 11:29:05-01:00  00035672a460b6eb5442d2837bc783f8af28c6f3   
0 2021-06-08 09:46:09-01:00  00e7e12a3a412ea386806d5d4eeaed345e912940   

                    repo                             filepath  \
0  django-rest-framework            rest_framework\schemas.py   
0                 poetry            poetry\inspection\info.py   
0                 scrapy     scrapy\contrib\pipeline\files.py   
0                 django  django\db\models\fields\__init__.py   
0                  black                 src\black\linegen.py   

                                             content  \
0  b'from importlib import import_module\n\nfrom ...   
0  b'import glob\nimport logging\nimport os\nimpo...   
0 

In [73]:
# Using DuckDB (memory-friendly for large parquet)
import duckdb
con = duckdb.connect()

q = f"""
SELECT repo, COUNT(*) as cnt
FROM read_parquet('{file_path}')
GROUP BY repo
ORDER BY cnt DESC
"""
repo_counts = con.execute(q).df()
print(repo_counts.head(30))

                     repo    cnt
0                 ansible  39060
1                  sentry  33506
2                    core  33292
3                     ray  16008
4                  pandas  14251
5                  django  10273
6               lightning  10169
7                 airflow   9105
8                 cpython   6422
9            scikit-learn   4571
10                 celery   3043
11                  spaCy   2262
12                 poetry   2083
13             localstack   1648
14           transformers   1644
15              openpilot   1439
16                 yolov5   1048
17                  numpy    909
18                    jax    665
19                 redash    642
20                  black    500
21                 scrapy    390
22  django-rest-framework    376
23                 pipenv    113
24              freqtrade      1


In [5]:
black_df = df[df['repo'] == 'black']


In [6]:
len(black_df)


500

In [7]:
content = black_df.iloc[1]['content'].decode('utf-8', errors='ignore')
print(content)


"""
Simple formatting on strings. Further string formatting code is in trans.py.
"""

import regex as re
import sys
from typing import List, Pattern


STRING_PREFIX_CHARS = "furbFURB"  # All possible string prefix characters.


def sub_twice(regex: Pattern[str], replacement: str, original: str) -> str:
    """Replace `regex` with `replacement` twice on `original`.

    This is used by string normalization to perform replaces on
    overlapping matches.
    """
    return regex.sub(replacement, regex.sub(replacement, original))


def has_triple_quotes(string: str) -> bool:
    """
    Returns:
        True iff @string starts with three quotation characters.
    """
    raw_string = string.lstrip(STRING_PREFIX_CHARS)
    return raw_string[:3] in {'"""', "'''"}


def lines_with_leading_tabs_expanded(s: str) -> List[str]:
    """
    Splits string into lines and expands only leading tabs (following the normal
    Python rules)
    """
    lines = []
    for line in s.splitlines():
        

In [8]:
lines = content.splitlines()
print("Total lines:", len(lines))

Total lines: 216


In [78]:
row = black_df.iloc[0]

print("Methods:")
print(row['methods'])

print("\nLines:")
print(row['lines'])


Methods:
['visit_STRING']

Lines:
[229 230 231]


In [79]:
content = black_df.iloc[0]['content'].decode('utf-8', errors='ignore')
lines_to_print = black_df.iloc[0]['lines']   # [229, 230, 231]

all_lines = content.splitlines()

for ln in lines_to_print:
    if 1 <= ln <= len(all_lines):
        print(f"{ln:04}: {all_lines[ln - 1]}")


0229:             docstring = normalize_string_prefix(leaf.value, self.remove_u_prefix)
0230:             prefix = get_string_prefix(docstring)
0231:             docstring = docstring[len(prefix) :]  # Remove the prefix


In [80]:
# Look at basic info of the original DataFrame
print(df.info())

# Look at basic info of the repo_counts DataFrame
print(repo_counts.info())

# Check descriptive statistics of 'cnt' from repo_counts
print(repo_counts['cnt'].describe())

# See unique values and distribution of 'cnt' from repo_counts
print(repo_counts['cnt'].value_counts().head(20))

# The cross-check below is already performed by the DuckDB query that created repo_counts
# If you wanted to do this with pandas on the original df, you would need to group and count first
# print(df.groupby('repo')['repo'].count().reset_index(name='cnt'))

<class 'pandas.core.frame.DataFrame'>
Index: 193420 entries, 0 to 43564
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype                                
---  ------    --------------   -----                                
 0   datetime  193420 non-null  datetime64[us, pytz.FixedOffset(-60)]
 1   commit    193420 non-null  object                               
 2   repo      193420 non-null  object                               
 3   filepath  193420 non-null  object                               
 4   content   190904 non-null  object                               
 5   methods   193420 non-null  object                               
 6   lines     193420 non-null  object                               
dtypes: datetime64[us, pytz.FixedOffset(-60)](1), object(6)
memory usage: 11.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0  

In [81]:
import pandas as pd

# Assuming df is your full dataset
# Filter only rows where repo == "black"
black_df = df[df['repo'] == 'black']

# Inspect the shape (number of rows and columns)
print("Rows:", black_df.shape[0], "Columns:", black_df.shape[1])

# Show first few rows
print(black_df.head())

# If you want to see all unique commits for black
print("Unique commits:", black_df['commit'].nunique())

# If you want to see all filepaths involved
print("Unique filepaths:", black_df['filepath'].nunique())

Rows: 500 Columns: 7
                   datetime                                    commit   repo  \
0 2021-06-08 09:46:09-01:00  00e7e12a3a412ea386806d5d4eeaed345e912940  black   
1 2021-06-08 09:46:09-01:00  00e7e12a3a412ea386806d5d4eeaed345e912940  black   
2 2021-06-08 09:46:09-01:00  00e7e12a3a412ea386806d5d4eeaed345e912940  black   
3 2018-05-28 17:53:54-01:00  023e61a2545b70750d47fe31ac5265ffced16a0c  black   
4 2019-03-14 18:09:10-01:00  026c81b83454f176a9f9253cbfb70be2c159d822  black   

               filepath                                            content  \
0  src\black\linegen.py  b'"""\nGenerating lines of code.\n"""\nfrom fu...   
1  src\black\strings.py  b'"""\nSimple formatting on strings. Further s...   
2    src\black\trans.py  b'"""\nString transformers that can split and ...   
3              black.py  b'import asyncio\nimport pickle\nfrom asyncio....   
4              black.py  b'import asyncio\nfrom asyncio.base_events imp...   

                             

In [82]:
# Count how many times each filepath appears
filepath_counts = black_df['filepath'].value_counts()

# Show only filepaths that appear more than once
duplicates = filepath_counts[filepath_counts > 1]

print(duplicates)

# 👉 Count total unique files in this repo
num_files = black_df['filepath'].nunique()
print("Total unique files in black repo:", num_files)

filepath
black.py                             171
src\black\__init__.py                 72
setup.py                              37
src\black\linegen.py                  18
docs\conf.py                          15
blib2to3\pgen2\tokenize.py            12
src\black\mode.py                     12
src\black\trans.py                    11
src\black\parsing.py                   9
src\black\files.py                     9
src\black\nodes.py                     9
src\black\comments.py                  8
blib2to3\pgen2\driver.py               7
blib2to3\pgen2\grammar.py              6
src\blib2to3\pgen2\parse.py            6
src\blib2to3\pytree.py                 5
src\black_primer\cli.py                5
src\black\handle_ipynb_magics.py       5
blackd.py                              5
src\black\lines.py                     5
src\black_primer\lib.py                5
blib2to3\pygram.py                     4
src\black\cache.py                     3
src\black\strings.py                   3
src\bla

In [83]:
import pandas as pd
import os

# Paths
DRIVE_FOLDER = "/content/drive/MyDrive/defectors/line_bug_prediction_splits/random"
TRAIN_PARQUET = os.path.join(DRIVE_FOLDER, "train.parquet.gzip")

# Load
df_train = pd.read_parquet(TRAIN_PARQUET)

In [84]:
import pandas as pd
import os

# Paths
DRIVE_FOLDER = "/content/drive/MyDrive/defectors/line_bug_prediction_splits/random"
VAL_PARQUET   = os.path.join(DRIVE_FOLDER, "val.parquet.gzip")

# Load
df_val   = pd.read_parquet(VAL_PARQUET)

In [85]:
import pandas as pd
import os

# Paths
DRIVE_FOLDER = "/content/drive/MyDrive/defectors/line_bug_prediction_splits/random"
TEST_PARQUET  = os.path.join(DRIVE_FOLDER, "test.parquet.gzip")

# Load
df_test  = pd.read_parquet(TEST_PARQUET)

In [86]:
print("Train:", df_train.shape, "Val:", df_val.shape, "Test:", df_test.shape)

Train: (193420, 7) Val: (10000, 7) Test: (10000, 7)


In [87]:
#Filter black reposotiory

df_val_black   = df_val[df_val["repo"] == "black"]
df_test_black  = df_test[df_test["repo"] == "black"]
df_train_black = df_train[df_train["repo"] == "black"]   # ✅ fixed


In [88]:
# Sanity check
print("Val - black rows:", len(df_val_black))
print("Test - black rows:", len(df_test_black))
print("Train - black rows:", len(df_train_black))

Val - black rows: 29
Test - black rows: 27
Train - black rows: 500


In [89]:
pd.DataFrame({
    "Validation": df_val_black.columns,
    "Test": df_test_black.columns,
    "Train": df_train_black.columns
})


,Validation,Test,Train
0,datetime,datetime,datetime
1,commit,commit,commit
2,repo,repo,repo
3,filepath,filepath,filepath
4,content,content,content
5,methods,methods,methods
6,lines,lines,lines


In [90]:
#Rename the column
df_val_black   = df_val_black.rename(columns={'lines': 'induce_bug'})
df_test_black  = df_test_black.rename(columns={'lines': 'induce_bug'})
df_train_black = df_train_black.rename(columns={'lines': 'induce_bug'})


In [91]:
print(df_val_black.columns.tolist())
print(df_test_black.columns.tolist())
print(df_train_black.columns.tolist())


['datetime', 'commit', 'repo', 'filepath', 'content', 'methods', 'induce_bug']
['datetime', 'commit', 'repo', 'filepath', 'content', 'methods', 'induce_bug']
['datetime', 'commit', 'repo', 'filepath', 'content', 'methods', 'induce_bug']


In [92]:
import pandas as pd
import os

OUTPUT_DIR   = "/content/drive/MyDrive/black_only"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save as CSV
df_val_black.to_csv(
    os.path.join(OUTPUT_DIR, "val_black.csv"),
    index=False
)

df_test_black.to_csv(
    os.path.join(OUTPUT_DIR, "test_black.csv"),
    index=False
)

df_train_black.to_csv(
    os.path.join(OUTPUT_DIR, "train_black.csv"),
    index=False
)

In [93]:
import duckdb

con = duckdb.connect()

VAL_PATH  = "/content/drive/MyDrive/defectors/line_bug_prediction_splits/random/val.parquet.gzip"
TEST_PATH = "/content/drive/MyDrive/defectors/line_bug_prediction_splits/random/test.parquet.gzip"

def repo_counts(parquet_path, split_name):
    q = f"""
    SELECT repo, COUNT(*) AS cnt
    FROM read_parquet('{parquet_path}')
    GROUP BY repo
    ORDER BY cnt DESC
    """
    df = con.execute(q).df()
    print(f"\n📌 Repo counts for {split_name}")
    print(df.head(30))
    return df

df_val_counts  = repo_counts(VAL_PATH, "VAL")
df_test_counts = repo_counts(TEST_PATH, "TEST")



📌 Repo counts for VAL
                     repo   cnt
0                 ansible  2187
1                  sentry  2010
2                    core  1306
3                     ray   888
4                  pandas   671
5               lightning   492
6                  django   491
7                 cpython   431
8                 airflow   429
9            scikit-learn   239
10                 celery   230
11                  spaCy    96
12           transformers    94
13             localstack    77
14                 poetry    74
15              openpilot    70
16                  numpy    44
17                 yolov5    38
18                 redash    37
19                  black    29
20                    jax    27
21                 scrapy    23
22  django-rest-framework    13
23                 pipenv     4

📌 Repo counts for TEST
                     repo   cnt
0                 ansible  2318
1                  sentry  1969
2                    core  1350
3                     ray